# Histopathologic Cancer Detection: Simple CNN comparison

**Jay Manvirk (Ivan Loginov)**<br/>University of Colorado, Boulder<br/>jay.manvirk@gmail.com

# Table of Contents

1. [Abstract](#chapter_1)
2. [Introduction](#chapter_2)
3. [Libraries and raw data](#chapter_3)
    - 3.1 [Libraries](#chapter_3_1)
    - 3.2 [Raw data](#chapter_3_2)
4. [Exploratory data analysis](#chapter_4)
    - 4.1 [Short datasets summary](#chapter_4_1)
    - 4.2 [Number of records per label](#chapter_4_2)
    - 4.3 [Images](#chapter_4_3)
5. [Data preprocessing](#chapter_5)
    - 5.1 [Sampling](#chapter_5_1)
    - 5.2 [Train-test split](#chapter_5_2)
    - 5.3 [Parallel preprocessing](#chapter_5_3)
6. [Model architecture](#chapter_6)
    - 6.1 [Base model](#chapter_6_1)
    - 6.2 [Dropout and batch normalization](#chapter_6_2)
    - 6.3 [Tuned hyperparameters](#chapter_6_3)
7. [Model results comparison](#chapter_7)
    - 7.1 [Base model results](#chapter_7_1)
    - 7.2 [Dropout and batch normalization results](#chapter_7_2)
    - 7.3 [Tuned hyperparameters results](#chapter_7_3)
    - 7.4 [Table results comparison](#chapter_7_4)
8. [Submission Results](#chapter_8)
9. [Conclusion](#chapter_9)
10. [References](#chapter_10)

# 1. Abstract <a class="anchor" id="chapter_1"></a>

Short summary. Goal. Results.

# 2. Introduction <a class="anchor" id="chapter_2"></a>

In this notebook we're going to look at 3 simple CNN models' performance comparison:

* Base model, which is going to include several basic layers
* Regularized and normalized model, where we implement Dropout and Batch Normalization layers
* Tuned model, layers of which are going to be the same as in the previous one, but with different parameters

For the sake of boosting the computational training speed of the aforementioned models we're going to incorporate Tensorflow parallelism and GPU accelerators.

This study is mainly focused on the brief analysis of the hyperparameters' impact on the model performance. It's going to be divided into several sections:
* **Exploratory Data Analysis:**<br/>
As was mentioned in the preface of this dataset, the data has been already modified, so that it is duplicate-free. However, it's not enough to accept that data doesn't need any additional manipulation. Therefore we're going to look closer at the dataset internals for any additional cleaning or type conversion. Also we will examine the number of records per label to see whether the dataset needs sampling or not.
* **Data preprocessing:**<br/>
Here we will introduce 3 steps to improve future model trainings efficiency. Sampling, train-test split and parallel preprocessing.<br/>
In the Sampling section we will try to even number of records per label for better label representation during training. The train-test split section is presented only due to the reason, that we're not going to use cross validation. We will manually pass train and test sets directly to the models' fit function. And finally the first parallelism would be introduced in the parallel preprocessing section. It's going to be about how to utilize CPU cores efficiently to load images as fas as possible.
* **Model Architecture:**<br/>
There will be an incremental update in every model except the base one. In essence they are supposed to be simple and straightforward, so that it's easier to spot the impact of the introduced updates and that this notebook doesn't take hours to run.
* **Model results comparison:**<br/>
This section is going to be filled with train/test scores per epoch charts and final table with the fitted models' runtime and evaluated ROC AUC scores.

# 3. Libraries and data <a class="anchor" id="chapter_3"></a>

## 3.1 Libraries <a class="anchor" id="chapter_3_1"></a>

In [ ]:
# basics
import os
import time
import numpy as np

# EDA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data preprocessing
import tensorflow as tf
import tensorflow_io as tfio
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Convolutional neural network
from keras.models import Sequential
from tensorflow.keras import layers, models
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam

# helper functions
from PIL import Image
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import load_model

## 3.2 Raw Data <a class="anchor" id="chapter_3_2"></a>

In [ ]:
input_dir = '/kaggle/input/histopathologic-cancer-detection'
list_l = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
list_l

In [ ]:
sample_data = pd.read_csv(list_l[0])
train_data = pd.read_csv(list_l[1])
train_dir = list_l[3] + '/'
test_dir = list_l[2] + '/'

# 4. Exploratory data analysis <a class="anchor" id="chapter_4"></a>

## 4.1 Short datasets summary <a class="anchor" id="chapter_4_1"></a>

In [ ]:
def print_short_summary(name, data):
    """
    Prints data head, shape and info.
    Args:
        name (str): name of dataset
        data (dataframe): dataset in a pd.DataFrame format
    """
    print(name)
    print('\n1. Data head:')
    print(data.head())
    print('\n2. Data shape: {}'.format(data.shape))
    print('\n3. Data info:')
    data.info()
    
def print_number_files(dirpath):
    print('{}: {} files'.format(dirpath, len(os.listdir(dirpath))))

In [ ]:
print_short_summary('Train data', train_data)

In [ ]:
print_short_summary('Sample data', sample_data)

In [ ]:
print_number_files(train_dir)

In [ ]:
print_number_files(test_dir)

## 4.2 Number of records per label <a class="anchor" id="chapter_4_2"></a>

The number of records per label differs signinficantly. This might create a problem in the model evaluation using ROC AUC score, if only one class is present in the dataset. We will address this scenario in the 5.1 Sampling section.

In [ ]:
# Plot horizontal barplot of number of records per label
plt.figure(figsize=(16, 9))
tmp = train_data['label'].value_counts()
sns.barplot(y=['No Cancer', 'Cancer'], x=tmp.values, orient='h')
plt.xlabel('Number of records')
plt.ylabel('Label')
plt.title('Number of records per label')
plt.show()

## 4.3 Images <a class="anchor" id="chapter_4_3"></a>

In [ ]:
def get_images_to_plot(file_names):
    """
    Returns list of images
    Args:
        file_names: list of filenames
    Returns:
        list of image objects
    """
    return [Image.open(f) for f in file_names]

def get_image_label(dirname, data, labels, n = 5):
    dict_img = {}
    for l in labels:
        indexes = data['label'] == l
        tmp = data[indexes][:n]
        tmp = dirname + tmp['id'] + '.tif'
        tmp = tmp.values
        tmp = get_images_to_plot(tmp)
        dict_img[l] = tmp
        
    return dict_img

In [ ]:
# Print original image size
img_path = train_dir + train_data['id'][0] + '.tif'
img = Image.open(img_path)
print('Original image size: {}'.format(img.size))

In [ ]:
# Get 5 filenames per label
data = get_image_label(train_dir,train_data, [0,1])

In [ ]:
# Initialize subplots with 2 rows and 5 columns
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(16, 9))

# Loop through the selected images and display in the respective rows
labels = ['No Cancer', 'Cancer']
for i in range(10):
    row = i // 5
    col = i % 5
    axes[row, col].imshow(data[row][col])
    axes[row, col].set_title(labels[row])
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# 5. Data preprocessing <a class="anchor" id="chapter_5"></a>

## 5.1 Sampling <a class="anchor" id="chapter_5_1"></a>

Downsampling or upsampling are about bias-variance trade-off. In the first one we reduce the number of records from the majority class. In the second one we replicate existing instances of minority class. In both cases we're trying to even number of labels to:
* mitigate error of calculating ROC AUC score if there is only one label present in the selected set
* improve model performance, be it better generalization (upsample) or faster training (downsample)

In our case the choice falls in favor of downsampling. Since we're dealing with simple CNN comparison our aim here is not the top performance algorithm, but rather an analysis of which model is more efficient. Thus we need to tune not just the models but the training data as well in order to reduce computational time as much as possible. Of course, all of that has to be done without significant loss in the ROC AUC score.

In [ ]:
# Majority class
no_cancer = train_data[train_data['label'] == 0]
# Minority class
cancer = train_data[train_data['label'] == 1]

# Downsample majority class to match minority class
no_cancer_downsampled = resample(no_cancer,
                              replace=False, 
                              n_samples=len(cancer),
                              random_state=0)

balanced_train_data = pd.concat([no_cancer_downsampled, cancer])

# Shuffle train data for training
balanced_train_data = balanced_train_data.sample(frac=1, random_state=0).reset_index(drop=True)


## 5.2 Train-test split <a class="anchor" id="chapter_5_2"></a>

Since we're not going to rely on cross validation, we need to provide both train and test samples to CNN to measure overfitting during training. Thus we first split our train dataset into 75% and 25% of train and test data respectively based on the default values of function.

In [ ]:
# Get full path to image including extension
image_paths = train_dir + balanced_train_data['id'] + '.tif'
image_paths = image_paths.values

labels = balanced_train_data['label'].values

X_train, X_test, y_train, y_test = train_test_split(image_paths
                                                    , labels
                                                    , test_size = 0.25
                                                    , shuffle = True
                                                    , random_state = 0)

## 5.3 Parallel preprocessing <a class="anchor" id="chapter_5_3"></a>

If we were to load images sequentially one-by-one or in batches, we would lose significant amount of time. Therefore it's more efficient to run this process in parallel, so that most of the time would be spent on actual training. With that in mind we're going to incorporate Tensorflow parallel processing functions to use optimal number of CPU cores provided in this notebook.

Additional important measures to increase training speed are the image resolution reduction and pixel scaling.<br/>Given our dataset of 96x96px images we can shrink them to 32x32px. It's an arbitrary reduction found across other kernels on Kaggle, but it helps in our case of speed-score trade-off.<br/>
The justification for pixel scaling lies in the idea of preventing gradients from becoming too large (exploding gradients) or too small (vanishing gradients) during backpropagation. Having this kind of constraint in the gradient values improve model performance overall.

In [ ]:
def get_decoded_image(image_path, label=None):
    """
    Load and preprocess images using TensorFlow I/O.
    Decode image with 4 channels RGBA.
    Resize image to 32x32px.
    Scale pixels from 0 to 1.
    Args:
        image_path: path to TIFF image
        label (optional): true label from train data
    Returns:
        (img, label): for train data
        img: for test data
    """
    img = tf.io.read_file(image_path)
    img = tfio.experimental.image.decode_tiff(img)
    img = tf.image.resize(img, [32, 32])
    img = tf.cast(img, tf.float32) / 255.0
    
    return img if label is None else (img, label)

def get_prefetched_data(data, batch_size, buffer_size):
    """
    Create a TensorFlow dataset from image paths and labels.
    Execution in parallel.
    Load, preprocess images, shuffle and batch the data.
    Prefetch batches to improve training performance.
    Args:
        data (tuple): image paths and corresponding labels
        batch_size (int): number of samples per batch
        buffer_size (int): number of elements from the dataset to buffer while shuffling
    Returns:
        tf.data.Dataset: preprocessed and preloaded TensorFlow dataset for keras CNN
    """
    # Autotune the degree of parallelism during training
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    
    # Create dataset from image paths and labels
    dataset = tf.data.Dataset.from_tensor_slices(data)

    # Apply parallel processing to load and preprocess images
    dataset = dataset.map(get_decoded_image, num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
# Get train and test datasets for optimal performance
BATCH_SIZE = 64
TRAIN_BUFFER_SIZE = X_train.shape[0]
TEST_BUFFER_SIZE = X_test.shape[0]

train_dataset = get_prefetched_data((X_train, y_train)
                                    , BATCH_SIZE
                                    , TRAIN_BUFFER_SIZE)
test_dataset = get_prefetched_data((X_test, y_test)
                                   , BATCH_SIZE
                                   , TEST_BUFFER_SIZE)

# 6. Model architecture <a class="anchor" id="chapter_6"></a>

## 6.1 Base model <a class="anchor" id="chapter_6_1"></a>

Our base model is going to be simple CNN yet with several layers. All the parameters are based either on the default values of a model, common values among other works or dataset features.
It is going to be comprised of:

1. **Convolutional Layers:**<br/>
It's said that Conv2D layer can capture spatial patterns in the input data. Ours is going to be with 32 filters of size (3, 3). The input shape (32, 32, 4) is used to address 32x32px images with 4 channels RGBA which were preprocessed by tfio.experimental.image.decode_tiff().
2. **Pooling Layers:**<br/>
The MaxPooling2D layer focuses on the most important features. Our setup is the window size of (2,2) and the stride of (2,2) which are the default values. With that we reduce the spatial dimensions of the feature map by half in both width and height.
3. **Flatten Layer:**<br/>
The Flatten layer transforms the 2D feature maps into a 1D vector, preparing the data for the subsequent dense layers.
4. **Dense Layers:**<br/>
The first Dense layer with 64 and 128 units and the ReLU activation function enables the model to learn intricate relationships in the data. The final Dense layer with 1 unit and the sigmoid activation function is to output probability of having cancer in the image.
7. **Activations:**<br/>
* ReLU: introduces non-linearity into the model by replacing all negative values in the input with zero and supposedly computationally efficient<br/>
* Sigmoid: within the range between 0 and 1 it represents the probability of belonging to the positive class, which is useful in our binary class problem

6. **Loss Function and Optimizer:**<br/>
* binary_crossentropy loss: forces the model to output probabilities close to 1 for cancer instances and close to 0 otherwise<br/>
* adam optimizer: efficient and common optimization algorithm


In [ ]:
def get_model_base():
    # Define base CNN
    model_base = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 4))
        , layers.MaxPooling2D((2, 2))

        , layers.Conv2D(64, (3, 3), activation='relu')
        , layers.MaxPooling2D((2, 2))

        , layers.Flatten()

        , layers.Dense(64, activation='relu')
        , layers.Dense(128, activation='relu')

        , layers.Dense(1, activation='sigmoid')
    ])
    
    return model_base

## 6.2 Dropout and batch normalization <a class="anchor" id="chapter_6_2"></a>

To tackle the problem of overfitting, we can add additional layer of Dropout Regularization. Dropout randomly sets a fraction of input units to 0 during training, which prevents model over adjusting its weights to train data. We're going to arbitrary set the dropout rate to 0.25, indicating that during training, 25% of the input units will be dropped out (set to 0).

To help solve another issue, slow training speed, we can add a layers of Batch Normalization. And thus normalize the activations of layers, making the optimization more efficient.

In [ ]:
def get_model_drop_bn():
    # Define CNN with dropout and batch normalization layers
    model_drop_bn = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 4))
        , layers.BatchNormalization()
        , layers.MaxPooling2D((2, 2))

        , layers.Conv2D(64, (3, 3), activation='relu')
        , layers.BatchNormalization()
        , layers.MaxPooling2D((2, 2))

        , layers.Flatten()

        , layers.Dense(64, activation='relu')
        , layers.Dense(128, activation='relu')

        , layers.Dropout(0.25)

        , layers.Dense(1, activation='sigmoid')
    ])
    
    return model_drop_bn

## 6.3 Tuned hyperparameters <a class="anchor" id="chapter_6_3"></a>

In this section we're going to tune several hyperparameters based on the previous network architecture. Concretely:
1. **Number of filters in the Conv2D layer:**<br/>
Increasing the number of filters can allow the model to capture more complex patterns in the data. We're going to double the amount of each.
2. **MaxPooling2d stride scale:**<br/>
Smaller stride presents a trade-off of more details for increased computation, while larger stride offers the opposite. We're going to decrease stride from (2,2) to (1,1) to increase model ROC AUC score.
3. **Dropout rate:**<br/>
The higher dropout rate the lower chances of overfitting. We're going to increase rate from 0.25 to 0.3.


In [ ]:
def get_model_tuned():
    # Define CNN with tuned hyperparameters
    model_tuned = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 4))
        , layers.BatchNormalization()
        , layers.MaxPooling2D((2, 2), strides = (1,1))

        , layers.Conv2D(128, (3, 3), activation='relu')
        , layers.BatchNormalization()
        , layers.MaxPooling2D((2, 2), strides = (1,1))

        , layers.Flatten()

        , layers.Dense(64, activation='relu')
        , layers.Dense(128, activation='relu')

        , layers.Dropout(0.3)

        , layers.Dense(1, activation='sigmoid')
    ])
    
    return model_tuned

# 7. Model results comparison <a class="anchor" id="chapter_7"></a>

In [ ]:
def roc_auc_score_(y_true, y_pred):
    """
    Calculate ROC AUC score using sklearn built-in function.
    Used in a model.compile as a custom metric.
    Args:
        y_true: true labels
        y_pred: predicted labels
    Returns:
        ROC AUC score (float)
    """
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float64)

def get_gpu_model(func):
    """
    Create model to be trained with a multi-GPU strategy.
    Args:
        func: function to get model architecture
    Returns:
        compiled_model: tensorflow model that performs data parallelism
                            by copying all of the model's variables
                            to each processor
    """
    # Check if GPU is available
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        # Create a MirroredStrategy.
        strategy = tf.distribute.MirroredStrategy()

        print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
        print('No GPU available, falling back to CPU.')

    with strategy.scope():
        compiled_model = func()
        compiled_model.compile(optimizer='adam'
                              , loss='binary_crossentropy'
                              , metrics=['accuracy'])

    return compiled_model

In [ ]:
def plot_model_scores(scores, model_name):
    """
    Plot train and test ROC AUC scores of a model by epoch
    """
    train_scores, test_scores = scores
    epochs = range(1, len(train_scores) + 1)

    # Plot train and test scores
    plt.figure(figsize=(16, 9))
    plt.plot(epochs, train_scores, label='Train score')
    plt.plot(epochs, test_scores, label='Test score')
    plt.title('Train and test accuracy scores of {}'.format(model_name))
    plt.xlabel('Epoch')
    plt.ylabel('ROC AUC Score')
    plt.legend()
    plt.grid(True)
    plt.show()

    
def get_model_results(model):
    """
    Return tuple of runtime, train and test scores.
    Compile, fit and save model along the way.
    Args:
        model: fitted model
    Returns:
        (runtime, (train_scores, test_scores) )
    """
    model = get_gpu_model(model)
    
    st = time.time()
    model.fit(train_dataset, epochs=5, validation_data=test_dataset)
    runtime = time.time() - st
    
    model.save('{}.h5'.format(model_name))
    
    train_scores = model.history.history['accuracy']
    test_scores = model.history.history['val_accuracy']
    
    del model
    
    return (runtime, (train_scores, test_scores))

## 7.1 Base model results <a class="anchor" id="chapter_7_1"></a>

In [ ]:
# Get train and test scores of every epoch
runtime_base, scores_base = get_model_results(get_model_base)

In [ ]:
# Plot scores
plot_model_scores(scores_base, 'base model')

## 7.2 Dropout and batch normalization results <a class="anchor" id="chapter_7_2"></a>

In [ ]:
# Get train and test scores of every epoch
runtime_drop_bn, scores_drop_bn = get_model_results(get_model_drop_bn)

In [ ]:
# Plot scores
plot_model_scores(scores_drop_bn, 'regularized and normalized model')

## 7.3 Tuned hyperparameters results <a class="anchor" id="chapter_7_3"></a>

In [ ]:
# Get train and test scores of every epoch
runtime_tuned, scores_tuned = get_model_results(get_model_tuned)

In [ ]:
# Plot scores
plot_model_scores(scores_tuned)

## 7.4 Table results comparison <a class="anchor" id="chapter_7_4"></a>

In [ ]:
table = [
    {
        'model':'Base'
        , 'runtime': runtime_base
        , 'train_roc_auc_score': scores_base[0][-1]
        , 'test_roc_auc_score': scores_base[1][-1]
    }
    ,{
        'model':'Drop and BN'
        , 'runtime': runtime_drop_bn
        , 'train_roc_auc_score': scores_drop_bn[0][-1]
        , 'test_roc_auc_score': scores_drop_bn[1][-1]
    }
    ,{
        'model':'Tuned'
        , 'runtime': runtime_tuned
        , 'train_roc_auc_score': scores_tuned[0][-1]
        , 'test_roc_auc_score': scores_tuned[1][-1]
    }
]

pd.DataFrame(table).sort_values(by = ['test_roc_auc_score','runtime']
                                , ascending = [False, True])

# 8. Submission Results <a class="anchor" id="chapter_8"></a>

After loading model with tuned hyperparaters as the top one trained on 20% of the dataset, it predicted new labels with ROC AUC score of **0.5012**.
As can be clearly seen its predictive ability is just a fraction better than simply flipping a coin.

In [ ]:
# Load save tuned model with custom metric parameter
model_20 = load_model('drop_bn.h5'
                           , custom_objects = {'roc_auc_score_': roc_auc_score_})

In [ ]:
# Create prefethed dataset of images to classify
submis_data = test_dir + sample_data['id'] + '.tif'
submis_data = submis_data.values

BATCH_SIZE = 64
SUBMIS_BUFFER_SIZE = submis_data.shape[0]

submis_dataset = get_prefetched_data((submis_data)
                                    , BATCH_SIZE
                                    , SUBMIS_BUFFER_SIZE)


In [ ]:
# Set predictions to result_20
result_20 = model_20.predict(submis_dataset)

In [ ]:
# Create table of ids and labels like sample_submission
sample_data['label'] = np.ravel(np.round(result_20))

In [ ]:
# Print submission table
sample_data

In [ ]:
# Make submission
sample_data.to_csv('submission_20.csv', index=False)

# 9. Conclusion <a class="anchor" id="chapter_9"></a>

Short analysis of results. Possible explanation. Use in production.

# 10. References <a class="anchor" id="chapter_10"></a>

* Better performance with the tf.data API<br/>
https://www.tensorflow.org/guide/data_performance
